In [ ]:
%matplotlib qt

In [ ]:
from main import *
import math

def constant_h(c: float) -> LearningRateFunc:
    return lambda k: c

h0 = 1
def geometric_h() -> LearningRateFunc:
    return lambda k: h0 / 2**k

def exponential_decay(λ: float) -> LearningRateFunc:
    assert λ > 0
    return lambda k: h0 * math.exp(-λ * k)

def polynomial_decay(α: float, β: float) -> LearningRateFunc:
    assert α > 0
    assert β > 0
    return lambda k: 1/math.sqrt(k + 1) * (β * k + 1) ** -α


In [ ]:
q1 = Quadratic(
    np.array([[1, 0], [0, 1]]),
    np.array([1, 1]),
    -1.4
)

q2 = Quadratic(
    np.array([[0.1, 0], [0, 3]]),
    np.array([0, 0]),
    0
)

q3 = Quadratic(
    np.array([[4, 0], [0, 1]]),
    np.array([-1, 2]),
    1.5
)

q4 = Quadratic(
    np.array([[4, 1], [1, -4]]),
    np.array([0, 0]),
    0
)

def multimodal_function(x, y):
    # Function with multiple local minima and one global minimum
    #term1 = np.sin(x)
    term2 = -1.0 * np.exp(-(x**2 + y**2)/10)
    # term3 = 0.1 * (x**2 + y**2)
    return term2

def mf2(x, y):
    return (x**2 + y - 11)**2 + (x + y**2 -7)**2

f1 = BiFuncCallableWrapper(multimodal_function)
f2 = BiFuncCallableWrapper(mf2)

In [ ]:
import numpy as np
from main import *

func = f1
x_0 = np.array([5.0, 4.9])
h = constant_h(0.1)

def relative_x_condition(x: np.ndarray, prev: np.ndarray) -> bool:
    # ‖𝑥_{𝑘+1} − 𝑥_𝑘‖ < 𝜀(‖𝑥_{𝑘+1}‖ + 1)
    eps = 1e-9
    return bool(np.linalg.norm(x - prev) < eps * (np.linalg.norm(x) + 1))

def relative_f_condition(x: np.ndarray, prev: np.ndarray) -> bool:
    # ‖∇𝑓(𝑥_𝑘)‖^2 < 𝜀‖∇𝑓(𝑥_0)‖^2
    eps = 1e-9
    return bool(np.linalg.norm(func.gradient(x) ** 2) < eps * np.linalg.norm(func.gradient(x_0)) ** 2)

print(f1.gradient(x_0))
trajectory = learning_rate_scheduling(x_0, func, h, relative_f_condition)

In [ ]:
eps = 1e-9
trajectory = steepest_gradient_descent_dichotomy(x_0, func, eps, relative_x_condition)

In [ ]:
eps = 1e-9
trajectory = steepest_gradient_descent_armijo(x_0, func, eps, relative_x_condition)

In [ ]:
from scipy.optimize import minimize
minimize(
    func,
    x_0,
    method='CG',
    jac=func.gradient,
    options={'disp': True}
)

In [ ]:
from main import *
import matplotlib.pyplot as plt

# Create a meshgrid for the 3D plot
x = np.linspace(-10, 10, 100)
y = np.linspace(-10, 10, 100)
X, Y = np.meshgrid(x, y)
Z = np.zeros_like(X)

for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        Z[i, j] = func(np.array([X[i, j], Y[i, j]]))

# Plot the 3D surface
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.6) # type: ignore

# Plot the trajectory
ax.plot(trajectory[:, 0], trajectory[:, 1], [func(np.array([x, y])) for x, y in trajectory], color='r', marker='o')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z') # type: ignore
plt.show()